In [9]:
import pandas as pd
import random as r
import math as m


basic = pd.read_csv('./data/battersBasic.csv')
clutch = pd.read_csv('./data/battersClutch.csv')
exp = pd.read_csv('./data/battersExp.csv')
PA = pd.read_csv('./data/battersPA.csv')
value = pd.read_csv('./data/battersValue.csv')
hit1 = pd.read_csv('./data/battersHit1.csv')
hit2 = pd.read_csv('./data/battersHit2.csv')
team1 = pd.read_csv('./data/battersTeambat1.csv')
team2 = pd.read_csv('./data/battersTeambat2.csv')

def sigmoid(average, vsAverage, num):
    return average - (average-vsAverage)/(1+m.exp(-0.5*(num-8)))

print(sigmoid(0.35, 0.25, 16))


#k, bb, 홈런, 내야, 외야_안타
    

0.25179862099620914


In [35]:
def hit_K_BB(name, outcount, base):
    playerK = exp.loc[exp["이름"] == name]['K%'] * 10
    playerBB = exp.loc[exp["이름"] == name]['BB%'] * 10
    playerhit = 1000 - playerK - playerBB
    
    selector = r.randrange(1,1001)
    
    if (selector <= int(playerK)) :
        outcount += 1
        return 'strikeout'
    elif (selector <= int(playerK + playerBB)) :
        base << 1
        base += 1
        return 'BB'
    else :
        return isHomerun(name)


In [36]:
def isHomerun(name):
    playerHit1 = hit1.loc[hit1["이름"] == name]
    playerHrNum = basic.loc[basic["이름"] == name]['홈런']
    playerInp = playerHit1['InP%']
    playerInp = playerInp.astype(float)
    playerPA = playerHit1['타석']
    playerHitNum = int(playerInp * playerPA / 100)
    playerHrRatio = playerHrNum/playerHitNum
    playerHrRatio *= 1000
    
    playerInfieldRatio = playerHit1['방향_내야%']
    playerInfieldRatio = playerInfieldRatio.astype(float)
    playerInfieldRatio *= 10
    

    
    playerOutfieldRatio = 1000 - playerInfieldRatio.values[0] - playerHrRatio.values[0]
    
    selector = r.randrange(1,1001)
    
    
    if(selector <= int(playerHrRatio)) :
        return 'homerun'
    elif(selector <= playerHrRatio.values[0] + playerInfieldRatio.values[0]):
        return infield(name)
    else:
        return outfield(name)
    

    return playerHrRatio, 100, playerOutfieldRatio, 100, playerInfieldRatio

In [37]:
def outfield(name):
    playerOutfieldBA = float(hit1.loc[hit1["이름"] == name]['타율_외'].values[0])
    
    selector = r.randrange(1,1001)
    playerOutfieldBA *= 1000
    if(selector <= playerOutfieldBA):
        return 'outfieldHit'
    else:
        return 'FO'
    

In [38]:
def outfieldFO(base, outcount,name):
    if(outcount == 2):
        outcount += 1
        return 'out'
    outcount += 1
    
    playerOutfieldCourse = float(hit2.loc[hit2["이름"] == name]['타구방향%_우'].values[0])
    playerOutfieldCourse *= 10
    
    course = 0
    selector = r.randrange(1,1001)
    
    if(selector <= playerOutfieldCourse):
        course = 1
    
    
    
    
    if(base & 4  == 4):
        base += 4
    if((base & 2 == 2) and (course)):
        base += 2
    
    return base
    

In [39]:
playerHitplus = team2.loc[team2["이름"] == '로하스']['H+_%']
print(playerHitplus)

43     38.5 
Name: H+_%, dtype: object


In [49]:
def outfieldHit(name, base):
    playerHitNum = int(basic.loc[basic["이름"] == name]['안타'].values[0])
    playerHitNum -= int(hit1.loc[hit1["이름"] == name]['내야안타_개수'].values[0])
    playerHitRight = float(hit2.loc[hit2["이름"] == name]['타구방향%_우'].values[0])
    playerHitLeft = float(hit2.loc[hit2["이름"] == name]['타구방향%_좌'].values[0])
    playerHitCenter = 100 - playerHitRight - playerHitLeft
    playerHitLeft *= 10
    playerHitRight *= 10
    playerHitCenter *= 10
    player2B = int(basic.loc[basic["이름"] == name]['2루타'].values[0])
    player3B = int(basic.loc[basic["이름"] == name]['3루타'].values[0])
    player1B = playerHitNum - player2B - player3B
    
    playerHitplus = float(team2.loc[team2["이름"] == name]['H+_%'].values[0])
    playerHitplus *= 10
    

    
    player1BRatio = 1000 * player1B/playerHitNum
    player2BRatio = 1000 * player2B/playerHitNum
    player3Bratio = 1000 * player3B/playerHitNum
    

    
    selector = r.randrange(1,1001)
    selector2 = r.randrange(1,1001)
    selector3 = r.randrange(1,1001)
    

    
    course = 0
    
    # course = 2 left, course = 1 right, course = 0 center
    
    if(selector2 <= playerHitLeft) :
        course = 2
    elif(selector2 <= playerHitLeft + playerHitRight) :
        course = 1
    else :
        course = 0
    
    
    if(selector <= int(player1BRatio)):
        if(selector3 <= int(playerHitplus)):
            base = base << 2
            print('1Bplus')
        else:
            base = base << 1
            print('1B')
        base += 1
        
    elif(selector <= int(player1BRatio + player2BRatio)):
        if(selector3 <= int(playerHitplus)):
            base = base << 3
            print('2Bplus')
        else:
            base = base << 2
            print('2B')
        base += 2
    else:
        base = base << 3
        base += 3
        print('3B')
    
    return base

In [51]:
# for i in range(100):
#     a = outfieldHit('로하스', i%8)
#     print(i)
#     print('init:' + str(i%8))
#     print('Result:'+str(a))

In [344]:
def infield(name) :
    
    playerInfieldFONum = int(hit1.loc[hit1["이름"] == name]['내야뜬공_개수'].values[0])
    playerHitInp = float(hit1.loc[hit1["이름"] == name]['InP%'].values[0])
    playerPA = int(hit1.loc[hit1["이름"] == name]['타석'].values[0])
    playerInfieldRatio = float(hit1.loc[hit1["이름"] == name]['방향_내야%'].values[0])
    playerHitNum = int(playerHitInp * playerPA / 100)
    playerInfieldNum = int(playerInfieldRatio * playerHitNum / 100)
    playerInfieldGround = playerInfieldNum - playerInfieldFONum
    playerInfieldFORatio = playerInfieldFONum/playerInfieldNum
    
    playerInfieldFORatio *= 1000
    
    selector = r.randrange(1,1001)
    
    if(selector <= playerInfieldFORatio):
        return 'FO'
    else:
        return 'groundball'
    
    

In [7]:
def groundBall(name,base,outcount,playerInfieldGround):
    
    playerInfieldHitNum = int(hit1.loc[hit1["이름"] == name]['내야안타_개수'].values[0])
    playerInfieldHitRatio = playerInfieldHitNum/playerInfieldGround
    playerInfieldHitRatio *= 1000
    print(playerInfieldHitRatio)
    playerForceOutNum = int(hit1.loc[hit1["이름"] == name]['포스o출루'].values[0])
    
    playerOutPlusRatio = float(team2.loc[team2["이름"] == name]['O+_%'].values[0])
    playerOutPlusRatio *= 10
    
    playerDORatio = float(team1.loc[team1["이름"] == name]['병살타%'].values[0])
    playerDORatio *= 10
    
    
    if outcount == 2 or base == 0:
        selector = r.randrange(1,1001)
        if(selector <= int(playerInfieldHitRatio)):
            base = base<<1
            base += 1
            return 'infieldHit'
        else :
            outcount += 1
            return 'GO'
    elif base & 1 == 1:
        selector = r.randrange(1,1001)
        if(selector <= int(playerInfieldHitRatio)):
            base = base<<1
            base += 1
            return 'infieldHit'
        else:
            selector2 = r.randrange(1,1001)
            if(selector2 <= int(playerOutPlusRatio)):
                base = base << 1
                outcount += 1
                return 'plus'
            else:
                outcount += 1
                return 'GO'
    else:
        selector = r.randrange(1,1001)
        if(selector <= int(playerInfieldHitRatio)):
            base = base<<1
            base += 1
            return 'infieldHit'
        elif(selector <= int(playerInfieldHitRatio + playerDORatio)):
            base = base<<1
            base -= 1
            outcount += 2
            return 'DoubleOut'
        else:
            selector2 = r.randrange(1,1001)
            if(selector2 <= int(playerOutPlusRatio)):
                base = base << 1
                outcount += 1
                return 'plus'
            else:
                outcount += 1
                return 'GO'
            
        
        

In [8]:
for i in range(100) :
    print(i)
    print(groundBall('로하스',5,1,100))
    print('')

0
80.0
plus

1
80.0
plus

2
80.0
GO

3
80.0
GO

4
80.0
GO

5
80.0
GO

6
80.0
infieldHit

7
80.0
plus

8
80.0
GO

9
80.0
plus

10
80.0
GO

11
80.0
infieldHit

12
80.0
plus

13
80.0
plus

14
80.0
GO

15
80.0
GO

16
80.0
GO

17
80.0
plus

18
80.0
GO

19
80.0
GO

20
80.0
GO

21
80.0
GO

22
80.0
GO

23
80.0
plus

24
80.0
GO

25
80.0
plus

26
80.0
GO

27
80.0
plus

28
80.0
infieldHit

29
80.0
plus

30
80.0
GO

31
80.0
plus

32
80.0
GO

33
80.0
GO

34
80.0
GO

35
80.0
GO

36
80.0
GO

37
80.0
GO

38
80.0
GO

39
80.0
infieldHit

40
80.0
GO

41
80.0
GO

42
80.0
GO

43
80.0
GO

44
80.0
GO

45
80.0
plus

46
80.0
GO

47
80.0
GO

48
80.0
GO

49
80.0
infieldHit

50
80.0
GO

51
80.0
GO

52
80.0
GO

53
80.0
GO

54
80.0
GO

55
80.0
GO

56
80.0
plus

57
80.0
plus

58
80.0
GO

59
80.0
GO

60
80.0
GO

61
80.0
GO

62
80.0
GO

63
80.0
GO

64
80.0
GO

65
80.0
infieldHit

66
80.0
GO

67
80.0
plus

68
80.0
GO

69
80.0
GO

70
80.0
plus

71
80.0
plus

72
80.0
GO

73
80.0
infieldHit

74
80.0
GO

75
80.0
GO

76
80.